In [1]:
from datasets import load_dataset
from tqdm import tqdm
import random
from datetime import datetime
import json
import os

os.system("mkdir -p out_data")
current_time_no_symbols = datetime.now().strftime(
    "%Y-%m-%d %H:%M:%S").replace("-", "").replace(":", "").replace(" ", "")
out_path = f"out_data/model_{current_time_no_symbols}.jsonl"


from vllm import LLM
import string
from vllm import SamplingParams
model_name="microsoft/Phi-3-medium-128k-instruct"
#model_name="OrionStarAI/Orion-14B-Chat"
llm = LLM(model=model_name,trust_remote_code=True,
          max_model_len=20000
          )

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:

#ds=load_dataset("kanhatakeyama/ChatbotArenaJaMixtral8x22b", split="train")
ds=load_dataset("wikipedia", "20220301.en",streaming=True,split="train")
#ds=load_dataset("hpprc/jawiki-wiktionary", split="train")

/home/setup/miniconda3/envs/llmeval/lib/python3.11/site-packages/datasets/load.py:1486: FutureWarning: The repository for wikipedia contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/wikipedia
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


In [12]:
try:
    ds.shuffle()
except:
    pass

In [13]:
import random

inst_dict={

"textbook":"""次のデータをもとに､論理的かつ教科書調の丁寧な日本語の文章を作成しなさい｡
-事実を正確に守り､推測出来ない事項については記述しないこと｡
-元の文章の流用は避け､表現や段落分け､文体などを必ず変更すること｡
-必ず日本語で出力すること

#データ
""",

"conversation":"""次のデータをもとに､論理的な日本語の会話文を作成しなさい｡
-事実を正確に守り､推測出来ない事項については記述しないこと｡
-元の文章の流用は避け､表現や段落分け､文体などを必ず変更すること｡
-必ず日本語で出力すること

#データ
""",
"logical":"""次のデータをもとに､論理的な文章を作成しなさい｡
-事実を正確に守り､推測出来ない事項については記述しないこと｡
-元の文章の流用は避け､表現や段落分け､文体などを必ず変更すること｡
-必ず日本語で出力すること

#データ
"""
}
def extract_random_part(text):
    text_length = len(text)
    extract_length = min(text_length, random.randint(400, 2000))
    start_index = random.randint(0, text_length - extract_length)
    return text[start_index:start_index + extract_length]

mode_list=list(inst_dict.keys())
n_records=5


In [14]:
cnt=0
loader=ds
def prepare_records(loader,n_records=10):

    records=[]
    cnt=0
    for record in loader:
        mode=random.choice(mode_list)
        inst=inst_dict[mode]
        text=record["text"]
        text=extract_random_part(text)
        text=f"""<|user|>
    {inst}{text}<|end|>
    <|assistant|>"""
        records.append(
            {"original_text":text,
            "mode":mode,
            "url":record["url"]
            }
                    )
        cnt+=1
        if cnt>n_records:
            break

    return records

In [15]:
while True:
    records=prepare_records(loader,n_records)
    prompts=[record["original_text"] for record in records]
    outputs = llm.generate(
            prompts,
            sampling_params = SamplingParams(
                temperature=0.1,
                max_tokens=1024,
                repetition_penalty=1.2,
            )
        )

    for record,output in zip(records,outputs):
        record["output_text"]=(output.outputs[0].text).strip()
        record.pop("original_text")
        with open(out_path, "a") as f:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")












Processed prompts: 100%|██████████| 12/12 [00:23<00:00,  1.98s/it]







Processed prompts: 100%|██████████| 6/6 [00:21<00:00,  3.59s/it]







Processed prompts: 100%|██████████| 6/6 [00:18<00:00,  3.06s/it]







Processed prompts: 100%|██████████| 6/6 [00:22<00:00,  3.83s/it]







Processed prompts: 100%|██████████| 6/6 [00:17<00:00,  2.84s/it]







Processed prompts: 100%|██████████| 6/6 [00:11<00:00,  1.94s/it]








KeyboardInterrupt: 